In [1]:
import pandas as pd

In [44]:
# load
data_df = pd.read_csv('datasets/mood_preprocessed_aggr_per_day_features.csv')
data_df['date'] = pd.to_datetime(data_df['date'])
data_df

date       id       screen  call  sms  appCat.builtin  \
0    2014-02-17  AS14.01     0.000000   2.0  0.0           0.000   
1    2014-02-17  AS14.02     0.000000   4.0  2.0           0.000   
2    2014-02-17  AS14.03     0.000000   0.0  1.0           0.000   
3    2014-02-17  AS14.06     0.000000   5.0  0.0           0.000   
4    2014-02-17  AS14.08     0.000000   0.0  4.0           0.000   
...         ...      ...          ...   ...  ...             ...   
1968 2014-06-05  AS14.24  1620.056999  11.0  0.0         801.994   
1969 2014-06-06  AS14.24  2169.781000  16.0  0.0         838.897   
1970 2014-06-07  AS14.24  1121.516999   0.0  2.0         171.285   
1971 2014-06-08  AS14.24   111.459000   0.0  0.0          88.239   
1972 2014-06-09  AS14.24     0.000000   0.0  0.0           0.000   

      appCat.communication  appCat.entertainment  appCat.finance  appCat.game  \
0                    0.000                   0.0             0.0          0.0   
1                    0.000                   0.0             0.0          0.0   
2                    0.000                   0.0             0.0          0.0   
3                    0.000                   0.0             0.0          0.0   
4                    0.000                   0.0             0.0          0.0   
...                    ...                   ...             ...          ...   
1968               795.073                   0.0             0.0          0.0   
1969              1205.799                   0.0             0.0          0.0   
1970               411.638                   0.0             0.0          0.0   
1971                 3.042                   0.0             0.0          0.0   
1972                 0.000                   0.0             0.0          0.0   

      ...  circumplex.arousal  circumplex.valence  activity  mood  weekday  \
0     ...           -1.000000                 0.0  0.071429   6.0        0   
1     ...            0.000000                 1.0  0.142857   7.0        0   
2     ...            1.000000                 2.0  0.290000   9.0        0   
3     ...           -1.000000                 0.0  0.120000   7.0        0   
4     ...            0.000000                 1.0  0.000000   7.0        0   
...   ...                 ...                 ...       ...   ...      ...   
1968  ...            0.223077                 1.0  0.253627   7.0        3   
1969  ...            0.702128                 1.0  0.287322   7.0        4   
1970  ...            0.333333                 1.0  0.317301   8.0        5   
1971  ...           -1.409091                 1.0  0.102054   7.0        6   
1972  ...           -1.000000                 1.0  0.250000   7.0        0   

      is_weekend  is_summer  is_winter  is_spring  is_autumn  
0              0          0          1          0          0  
1              0          0          1          0          0  
2              0          0          1          0          0  
3              0          0          1          0          0  
4              0          0          1          0          0  
...          ...        ...        ...        ...        ...  
1968           0          1          0          0          0  
1969           0          1          0          0          0  
1970           1          1          0          0          0  
1971           1          1          0          0          0  
1972           0          1          0          0          0  

[1973 rows x 27 columns]

In [45]:
def create_user_specific_windows_and_targets(data, target_column, window_size):
    windows = []
    targets = []
    
    # Iterate over each unique user to ensure windows are user-specific
    for user in data['id'].unique():
        user_data = data[data['id'] == user]
        
        for i in range(len(user_data) - window_size):
            end_ix = i + window_size
            
            # Ensure we do not go out of bounds when accessing the target
            if end_ix >= len(user_data):
                break

            # Extract the window and target, excluding the target column from the window
            window = user_data.iloc[i:end_ix].drop(columns=[target_column, 'id']).values.tolist()
            target = user_data.iloc[end_ix][target_column]
            windows.append(window)
            targets.append(target)

    return windows, targets



In [46]:
# window size
n = 5 

windows, targets = create_user_specific_windows_and_targets(data_df, 'mood', n)

In [47]:
# Function to split data into train and test sets
def train_test_split(data, train_size=0.8):
    split_index = int(len(data) * train_size)
    train_data = data[:split_index]
    test_data = data[split_index:]
    return train_data, test_data



In [48]:
# Apply the train-test split to both windows and targets
train_windows, test_windows = train_test_split(windows, 0.95)
train_targets, test_targets = train_test_split(targets, 0.95)


In [49]:
# Avoid leakage, delete last n-1 datapoints in training set
for _ in range(n-1):
    del train_targets[-1]
    del train_windows[-1]